In [46]:
#load library
import pandas as pd
import numpy as np

In [47]:
path="C:/Users/melika/Desktop/melika/ML_return_rate_prediction/csv/"

In [48]:
#load data
sales_yr1_cleaned=pd.read_csv(f"{path}cleaned_sales_yr1.csv",sep=",")
sales_yr2_cleaned=pd.read_csv(f"{path}cleaned_sales_yr2.csv",sep=",")
products_cleaned=pd.read_csv(f"{path}products_cleaned.csv",sep=",")


In [49]:
sales_yr1_cleaned.head()

,channel,date_id,pid,net_sales_amount,year,month
0,online,2022-02-01,b51a86b6eee6426,-1,2022,2
1,online,2021-04-02,669ae35300540ef,-1,2021,4
2,online,2020-04-10,52836da7b3f28e8,-1,2020,4
3,online,2021-04-11,e1a2f5ec090c1b7,1,2021,4
4,online,2019-05-01,e7fb7eb5f96c8cd,3,2019,5


In [50]:
products_cleaned.head(2)

,pid,brand,category,price,season,subcategory,subsubcategory,numeric_size,count_size
0,00018645b311291,StreamStride,Footwear,66.260,Winter,Adult,Fancy Feet,1,1
1,0002c09028f0997,ZenithMan,Sweaters,57.442,Summer,Adult,Round-Top,0,4


In [51]:
# Rename date_id
sales_yr1_cleaned.rename(columns={"date_id": "date"},inplace=True)
sales_yr2_cleaned.rename(columns={"date_id": "date"},inplace=True)

In [52]:
sales_yr1_cleaned.dtypes

channel             object
date                object
pid                 object
net_sales_amount     int64
year                 int64
month                int64
dtype: object

In [53]:
#Make sure dates are represented correctly
sales_yr1_cleaned["date"] = pd.to_datetime(sales_yr1_cleaned["date"], format='%Y-%m-%d')
sales_yr2_cleaned["date"] = pd.to_datetime(sales_yr2_cleaned["date"], format='%Y-%m-%d')

In [54]:
sales_yr1_cleaned.dtypes

channel                     object
date                datetime64[ns]
pid                         object
net_sales_amount             int64
year                         int64
month                        int64
dtype: object

In [55]:
print("min date year1",min(sales_yr1_cleaned["date"]))
print("max date year1",max(sales_yr1_cleaned["date"]))
print("min date year2",min(sales_yr1_cleaned["date"]))
print("max date year2",max(sales_yr1_cleaned["date"]))

min date year1 2018-01-12 00:00:00
max date year1 2023-04-11 00:00:00
min date year2 2018-01-12 00:00:00
max date year2 2023-04-11 00:00:00


In [56]:
# Extract the year and store it in a new column called 'year'
sales_yr1_cleaned['year'] = sales_yr1_cleaned['date'].dt.year
sales_yr2_cleaned['year'] = sales_yr2_cleaned['date'].dt.year
print(sales_yr1_cleaned['year'].value_counts())
print(sales_yr2_cleaned['year'].value_counts())


year
2021    261570
2022     91726
2020     27746
2023     10586
2018      8875
2019       770
Name: count, dtype: int64
year
2022    51712
2021     4346
2018     1381
2023     1140
2020      313
2019      106
Name: count, dtype: int64


In [57]:
# just keep year 2021 for year1 and 2022 for year2

sales_yr1_cleaned = sales_yr1_cleaned[sales_yr1_cleaned['year'] == 2021]
sales_yr2_cleaned = sales_yr2_cleaned[sales_yr2_cleaned['year'] == 2022]


In [59]:
# 01 dealing with y varialbe, yearly return rate
 
pid_sold_year1 = sales_yr1_cleaned[sales_yr1_cleaned['net_sales_amount'] > 0]
pid_sold_year1 = pid_sold_year1[['pid','net_sales_amount']]
pid_sold_year1= pid_sold_year1.groupby('pid')['net_sales_amount'].sum().reset_index()
pid_sold_year1.rename(columns={'net_sales_amount':"total_sell"} ,inplace=True)

pid_sold_year2 = sales_yr2_cleaned[sales_yr2_cleaned['net_sales_amount'] > 0]
pid_sold_year2 = pid_sold_year2[['pid','net_sales_amount']]
pid_sold_year2= pid_sold_year2.groupby('pid')['net_sales_amount'].sum().reset_index()
pid_sold_year2.rename(columns={'net_sales_amount':"total_sell"} ,inplace=True)

In [60]:
pid_return_year1 = sales_yr1_cleaned[sales_yr1_cleaned['net_sales_amount'] < 0]
pid_return_year1 = pid_return_year1[['pid','net_sales_amount']]
pid_return_year1= pid_return_year1.groupby('pid')['net_sales_amount'].sum().reset_index()
pid_return_year1.rename(columns={'net_sales_amount':"total_return"} ,inplace=True)

pid_return_year2 = sales_yr2_cleaned[sales_yr2_cleaned['net_sales_amount'] < 0]
pid_return_year2 = pid_return_year2[['pid','net_sales_amount']]
pid_return_year2= pid_return_year2.groupby('pid')['net_sales_amount'].sum().reset_index()
pid_return_year2.rename(columns={'net_sales_amount':"total_return"} ,inplace=True)

In [65]:
pid_return_rate_year1 = pid_return_year1.merge(pid_sold_year1, on='pid', how='right')
pid_return_rate_year1.fillna(value={'total_return':0},inplace=True)
# Assuming pid_return_rate_year1 is the DataFrame with 'total_return' and 'total_sell' columns
pid_return_rate_year1['yearly_return_rate'] = pid_return_rate_year1.apply(
    lambda row: 0 if row['total_sell'] == 0 else row['total_return'] * -1 / row['total_sell'],
    axis=1
)

pid_return_rate_year2 = pid_return_year2.merge(pid_sold_year2, on='pid', how='right')
pid_return_rate_year2.fillna(value={'total_return':0},inplace=True)
# Assuming pid_return_rate_year1 is the DataFrame with 'total_return' and 'total_sell' columns
pid_return_rate_year2['yearly_return_rate'] = pid_return_rate_year2.apply(
    lambda row: 0 if row['total_sell'] == 0 else row['total_return'] * -1 / row['total_sell'],
    axis=1
)


pid_return_rate_year1

,pid,total_return,total_sell,yearly_return_rate
0,00224c8caec6e96,-3.0,95,0.031579
1,002b22cf4d1247a,0.0,6,-0.000000
2,003f7b23afe2e4b,-14.0,96,0.145833
3,004231d36ee967c,-3.0,130,0.023077
4,004e81b565469e2,-1.0,1,1.000000
...,...,...,...,...
5822,ffcb12fb015cefb,-3.0,42,0.071429
5823,fff26be3fa7a676,0.0,6,-0.000000
5824,fffbef632ce3199,0.0,2,-0.000000
5825,fffeafe6012cca8,0.0,15,-0.000000
